# Combining blueprints and data projects for multi page documents

## Introduction

Data projects in Amazon Bedrock data automation (BDA) provide a easy way of grouping your standard and customt output configuration for processing files. You can create a BDA project and use the ARN of the project to call the `InvokeDataAutomationAsync` API. BDA processes the input file automatically using the configuration settings defined in that project. Output is then generated based on the project's configuration. You can use a single project resource for multiple file types. You can also configure a project with Blueprints to define custom output. 

When processing documents, you might want to use multiple blueprints for different kinds of documents that are passed to your project. BDA automatically matches your documents to the appropriate blueprint that's configured in your project, and generates custom output using that blueprint

You can also configure a project with Blueprints for documents (or images), to define custom output. In this notebook, we will explore the capability of using project with blueprints for processing documents. We will start with creating a project and associate with multiple blueprints based on kind of documents we expect to process.  We will process a file with a number of different document types and explore how BDA matched the document types in the file to appropriate blueprint and use that to to extract insights from the document.

You can configure custom output for documents by adding a new blueprint (or a preexisting blueprint from BDA global catalog) to the BDA project. If your use case has different kinds of documents then you can use  multiple blueprints for the different document kinds with the project.

**Note: A project chan have up to 40 document blueprints attached.**

When you attach multiple blueprints with a project, BDA would automatically find an appropriate blueprint matching using the input document. Once a matching blueprint is found, BDA generates custom output using that blueprint.

Let's go through the steps to creating a project and attaching a set of blueprints to process different file types.

## Prerequisites

In [ ]:
%pip install "boto3>=1.37.4" itables==2.2.4 pdf2image PyPDF2==3.0.1 --upgrade -qq

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup

Before we get to the part where we invoke BDA with our sample artifacts, let's setup some parameters and configuration that will be used throughout this notebook.

In [ ]:
import boto3
import json
import pprint
from IPython.display import JSON, display, IFrame, HTML
import sagemaker
import pandas as pd
from itables import show
import time
from utils import helper_functions
from utils import display_functions
from pathlib import Path
import os

session = sagemaker.Session()
default_bucket = session.default_bucket()
current_region = boto3.session.Session().region_name

sts = boto3.client('sts')
account_id = sts.get_caller_identity()['Account']

# Initialize Bedrock Data Automation client
bda_client = boto3.client('bedrock-data-automation')
bda_runtime_client = boto3.client('bedrock-data-automation-runtime')
s3_client = boto3.client('s3')

bda_s3_input_location = f's3://{default_bucket}/bda/input'
bda_s3_output_location = f's3://{default_bucket}/bda/output'

## Prepare sample document

For this lab, we use a sample `Bank Statement`. The statement contains account holder information, bank account details and balance summary along with a table of transactions. We will use a catalog blueprint with custom output to extract and analyse the document content.

In [ ]:
local_download_path = 'samples'
os.makedirs(local_download_path, exist_ok=True)
local_file_name = 'claims-pack.pdf'
source_url = 'https://ws-assets-prod-iad-r-pdx-f3b3f9f1a7d6a3d0.s3.us-west-2.amazonaws.com/335119c4-e170-43ad-b55c-76fa6bc33719/claims-pack.pdf'

!curl {source_url} --output {local_download_path}/{local_file_name}

In [ ]:


local_file_path = os.path.join(local_download_path, local_file_name)
#(bucket, key) = helper_functions.get_bucket_and_key(document_url)
#response = s3_client.download_file(bucket, key, local_file_path)

document_s3_uri = f'{bda_s3_input_location}/{local_file_name}'

target_s3_bucket, target_s3_key = helper_functions.get_bucket_and_key(document_s3_uri)
s3_client.upload_file(local_file_path, target_s3_bucket, target_s3_key)

print(f"Downloaded file to: {local_file_path}")
print(f"Uploaded file to S3: {target_s3_key}")
print(f"document_s3_uri: {document_s3_uri}")

### View Sample Document

In [ ]:
IFrame(local_file_path, width=1000, height=400)

### Create custom blueprints for our documents

Our sample file contains multiple document types. We would therefore use multiple blueprints to process the document. We will use some premade blueprint from the BDA blueprint global catalog. For other document types where we don't have an catalog blueprint, we would create a custom blueprint.

We use the `create_blueprint` operation (or `update_blueprint` to update an existing blueprint) in the  `boto3` API to create/update the blueprint. Each blueprint that you create is an AWS resource with its own blueprint ID and ARN. 

In [ ]:
# create blueprint using Boto3
blueprints = [
    {
        "name": 'claim-form',
        "description": 'Blueprint for Medical Claim form CMS 1500',
        "type": 'DOCUMENT',
        "stage": 'LIVE',
        "schema_path": 'blueprint-schemas/claims_form.json'
    },
    {
        "name": 'hospital-discharge-report',
        "description": 'Blueprint for Hospital discharge summary report',
        "type": 'DOCUMENT',
        "stage": 'LIVE',
        "schema_path": 'blueprint-schemas/discharge_summary.json'
    },
    {
        "name": 'medical-transcription',
        "description": ' Medical Transcription',
        "type": 'DOCUMENT',
        "stage": 'LIVE',
        "schema_path": 'blueprint-schemas/medical_transcription.json'
    },
    {
        "name": 'lab-reports',
        "description": ' Lab Reports',
        "type": 'DOCUMENT',
        "stage": 'LIVE',
        "schema_path": 'blueprint-schemas/lab_reports.json'
    },
    {
        "name": 'explanation-of-benefits',
        "description": 'Explantion of Benefits or a Medical Remittance Advice',
        "type": 'DOCUMENT',
        "stage": 'LIVE',
        "schema_path": 'blueprint-schemas/explanation_of_benefits.json'
    },
    
]


In [ ]:
blueprint_arns = []
for blueprint in blueprints:
    with open(blueprint['schema_path']) as f:
        blueprint_schema = json.load(f)
        blueprint_arn = helper_functions.create_or_update_blueprint(
            bda_client, 
            blueprint['name'], 
            blueprint['description'], 
            blueprint['type'],
            blueprint['stage'],
            blueprint_schema
        )
        blueprint_arns += [blueprint_arn]

The `update_data_automation_project` API takes a project name, description, stage (LIVE / DEVELOPMENT), the standard output configuration and a custom output configuration as input. We are only focussing on the custom output in this notebook, so we leave the standard output configuration as empty so BDA will use the defaults. Additionally, we use a custom configuration with the arn for the recommended blueprint.

### Create data project to process file with multiple document types
With custom blueprints created, we can now go ahead an create our data project. Check if the project with the given name already exists, if no create a new project with given configuration otherwise update the data project with the given configuration and stage.

A few points to note - 
* We have added multiple blueprints to our data project to match the document types we would expect to file in the claim pack

* Because our sample file contains multiple documents, we pass the `overrideConfiguration` to the api call, with `document splitter` enabled.  With this setting, BDA scans the file and splits it into individual documents based on context. Those individual documents are then matched to the correct blueprint for processing.

In [ ]:
bda_project_name = 'document-custom-output-multiple-blueprints'
bda_project_stage = 'LIVE'
standard_output_configuration = {
    'document': {
        'extraction': {
            'granularity': {
                'types': [
                    'DOCUMENT', 'PAGE'
                ]
            },
            'boundingBox': {
                'state': 'ENABLED'
            }
        },
        'generativeField': {
            'state': 'ENABLED'
        },
        'outputFormat': {
            'textFormat': {
                'types': [
                    'MARKDOWN'
                ]
            },
            'additionalFileFormat': {
                'state': 'ENABLED'
            }
        }
    }
}

custom_output_configuration = {
    "blueprints": [
        {
            'blueprintArn': 'arn:aws:bedrock:us-west-2:aws:blueprint/bedrock-data-automation-public-bank-statement',
            'blueprintStage': 'LIVE'
        },
        {
            'blueprintArn': 'arn:aws:bedrock:us-west-2:aws:blueprint/bedrock-data-automation-public-us-driver-license',
            'blueprintStage': 'LIVE'
        },
        {
            'blueprintArn': 'arn:aws:bedrock:us-west-2:aws:blueprint/bedrock-data-automation-public-invoice',
            'blueprintStage': 'LIVE'
        },
        {
            'blueprintArn': 'arn:aws:bedrock:us-west-2:aws:blueprint/bedrock-data-automation-public-prescription-label',
            'blueprintStage': 'LIVE'
        },
        {
            'blueprintArn': 'arn:aws:bedrock:us-west-2:aws:blueprint/bedrock-data-automation-public-us-medical-insurance-card',
            'blueprintStage': 'LIVE'
        }
    ]
}
custom_output_configuration['blueprints'] += [
    {
        'blueprintArn': blueprint_arn,
        'blueprintStage': 'LIVE'
    } for blueprint_arn in blueprint_arns
]

override_configuration={'document': {'splitter': {'state': 'ENABLED'}}}

In [ ]:
list_project_response = bda_client.list_data_automation_projects(
    projectStageFilter=bda_project_stage)

project = next((project for project in list_project_response['projects']
               if project['projectName'] == bda_project_name), None)

if not project:
    response = bda_client.create_data_automation_project(
        projectName=bda_project_name,
        projectDescription='Document processing combining blueprints with data projects',
        projectStage=bda_project_stage,
        standardOutputConfiguration=standard_output_configuration,
        customOutputConfiguration=custom_output_configuration,
        overrideConfiguration=override_configuration
    )
else:
    response = bda_client.update_data_automation_project(
        projectArn=project['projectArn'],
        standardOutputConfiguration=standard_output_configuration,
        customOutputConfiguration=custom_output_configuration,
        overrideConfiguration=override_configuration
    )

project_arn = response['projectArn']

### Wait for create/update data project operation completion

In [ ]:
status_response = helper_functions.wait_for_completion(
            client=bda_client,
            get_status_function=bda_client.get_data_automation_project,
            status_kwargs={'projectArn': project_arn},
            completion_states=['COMPLETED'],
            error_states=['FAILED'],
            status_path_in_response='project.status',
            max_iterations=15,
            delay=30
)

### Invoke Data Automation Async
With the data project configured, we can now invoke data automation for our sample document. When we submit the document for processing, BDA scans the file and splits it into individual documents based on contextand matches it against the list of blueprints provided.

In [ ]:
print(f"Invoking bda - input: {document_s3_uri}")
print(f"Invoking bda - output: {bda_s3_output_location}")

response = bda_runtime_client.invoke_data_automation_async(
    inputConfiguration={
        's3Uri': document_s3_uri
    },
    outputConfiguration={
        's3Uri': bda_s3_output_location
    },
    dataAutomationConfiguration={
        'dataAutomationProjectArn': project_arn,
        'stage': 'LIVE'
    },
    dataAutomationProfileArn=f'arn:aws:bedrock:{current_region}:{account_id}:data-automation-profile/us.data-automation-v1'
)


invocationArn = response['invocationArn']
print(f'Invoked data automation job with invocation arn {invocationArn}')

### Get Data Automation Status

We can check the status and monitor the progress of the Invocation job using the `GetDataAutomationStatus`. This API takes the invocation arn we retrieved from the response to the `InvokeDataAutomationAsync` operation above.

The invocation job status moves from `Created` to `InProgress` and finally to `Success` when the job completes successfully, along with the S3 location of the results. If the job encounters and error the final status is either `ServiceError` or `ClientError` with error details

In [ ]:
status_response = helper_functions.wait_for_completion(
            client=bda_client,
            get_status_function=bda_runtime_client.get_data_automation_status,
            status_kwargs={'invocationArn': invocationArn},
            completion_states=['Success'],
            error_states=['ClientError', 'ServiceError'],
            status_path_in_response='status',
            max_iterations=15,
            delay=30
)
if status_response['status'] == 'Success':
    job_metadata_s3_location = status_response['outputConfiguration']['s3Uri']
else:
    raise Exception(f'Invocation Job Error, error_type={status_response["error_type"]},error_message={status_response["error_message"]}')

### View Job Metadata
Let's retrieve the job metadata. The Job metadata contains the S3 uri's for the standard output,custom output and the status of custom output. The custom output status could be either of `MATCH` or `NO_MATCH`. `MATCH` indicates BDA was able to find a matching blueprint for the specific segment from the list of blueprint we associated with the project. If BDA was unable to match the segment to a blueprint associated with the project then the `custom output status` is `NO_MATCH` and in this case BDA would only have a standard output extracted from that specific segment of the input file.

In [ ]:
job_metadata = json.loads(helper_functions.read_s3_object(job_metadata_s3_location))

job_metadata_table = pd.DataFrame(job_metadata['output_metadata'][0]['segment_metadata']).fillna('')
job_metadata_table.index.name='Segment Index'
job_metadata_json = JSON(job_metadata, root="job_metadata", expanded=True)
# Display the widget
display_functions.display_multiple(
    [display_functions.get_view(job_metadata_table), display_functions.get_view(job_metadata_json)], 
    ["Table View", "Raw JSON"])

### View Segments and Matched Blueprints
As we can see in the `job metadata`, BDA creates a segment section each for each individual document that it has identified in the file. Each segment section has details on the matched blueprint and the results of the extraction. For each segment, BDA also outputs the page indices (one or more) from the original file.

We can now get the custom output corresponding to each segment and look at the insights that BDA custom output produces.

In [ ]:
asset_id = 0
segments_metadata = next(item["segment_metadata"]
                                for item in job_metadata["output_metadata"] 
                                if item['asset_id'] == asset_id)

standard_outputs = [
    json.loads(helper_functions.read_s3_object(segment_metadata.get('standard_output_path')))for segment_metadata in segments_metadata]
custom_outputs = [json.loads(helper_functions.read_s3_object(segment_metadata.get('custom_output_path'))) if segment_metadata.get('custom_output_status') == 'MATCH' else None for segment_metadata in segments_metadata]

### View Custom output summary

In [ ]:
custom_outputs_json = JSON(custom_outputs, root="custom_outputs", expanded=False)
custom_outputs_table = pd.DataFrame(helper_functions.get_summaries(custom_outputs)).fillna('')

display_functions.display_multiple(
    [
        display_functions.get_view(custom_outputs_table.style.hide(axis='index')),
        display_functions.get_view(custom_outputs_json)
    ], 
    ["Table View", "Raw JSON"])

### Explore Document Insights using Standard and Custom output

In [ ]:
views=[]
titles=[]
# Use the function
for custom_output, standard_output in zip(custom_outputs, standard_outputs):
    if custom_output:
        result = helper_functions.transform_custom_output(custom_output['inference_result'], custom_output['explainability_info'][0])
        document_image_uris = [page.get('asset_metadata',{}).get('rectified_image') for page in standard_output.get('pages',[])]
        views += [display_functions.segment_view(document_image_uris=document_image_uris,
                   inference_result=result)]
        titles += [custom_output.get('matched_blueprint', {}).get('name', None)]
display_functions.display_multiple(views, titles)

## Summary & Next Steps

In this lab we explored how we can leverage the versatility of blueprints along with data projects to extract customized output from multiple documents.

For a deeper dive into intelligent document processing, try our workshop on `Document processing with Amazon Bedrock Data Automation` where you can explore creating and using blueprints with various IDP goals such as Classification, Normalization, Transformation etc., along with end-to-end industry use cases.

## Clean Up
Let's delete uploaded sample file from s3 input directory and the generated job output files.

In [ ]:
import os
from pathlib import Path


# Delete S3 File
s3_client.delete_object(Bucket=target_s3_bucket, Key=target_s3_key)

# Delete local file
if os.path.exists(local_file_path):
    os.remove(local_file_path)	

# Delete bda job output
bda_s3_job_location = str(Path(job_metadata_s3_location).parent).replace("s3:/","s3://")
!aws s3 rm {bda_s3_job_location} --recursive